In [164]:
import math
from collections import Counter
import os
import numpy as np
import pandas as pd

In [165]:
os.chdir("E:/AI/assignment_4")

In [166]:
data=pd.read_csv("challenge_train.csv")

In [167]:
data=data.values

In [168]:
np.random.shuffle(data)

In [170]:
test_size=int(data.shape[0]*0.2)
train_size=int(data.shape[0]*0.8*0.8)
valid_size=int(data.shape[0])-test_size-train_size

In [171]:
test_data=data[:test_size]

In [200]:
test_X=test_data[:,1:]
test_Y=test_data[:,0]

In [172]:
total_train_data=data[test_size:]

In [173]:
class treenode():
    def __init__(self,left=None,right=None,function=None,label=None):
        self.left=left
        self.right=right
        self.function=function
        self.label=label
    def decide_label(self,feature):
        if self.label is not None:
            return self.label
        elif self.function(feature):
            return self.left.decide_label(feature)
        else:
            return self.right.decide_label(feature)

In [174]:
def Gini(labels):
    unique_elements, counts_elements = np.unique(labels, return_counts=True)
    p=counts_elements[0]/counts_elements.sum()
    return p*(1-p)

In [175]:
def GiniGain(labels,leftlabel,rightlabel):
    if leftlabel.size==labels.size or rightlabel.size==labels.size:
        return 0
    else:
        original_gini=Gini(labels)
        left_gini=Gini(leftlabel)
        right_gini=Gini(rightlabel)
        return original_gini-leftlabel.size/labels.size*left_gini-rightlabel.size/labels.size*right_gini

In [293]:
class DecisionTree():
    def __init__(self,max_depth=10):
        self.max_depth=max_depth
    def Gini_split(self,data,label_collum=0):
        labels=data[:,label_collum]
        features=np.delete(data,label_collum,axis = 1) 
        max_ginigain=-float("inf")
        for i in range(features.shape[1]):
            for j in [25,50,75]:
                split_point=np.percentile(features[:,i],j)
                leftlabel=labels[features[:,i]<split_point]
                rightlabel=labels[features[:,i]>=split_point]
                #print("leftlabel,rightlabel: ", leftlabel, rightlabel)
                gini_gain=GiniGain(labels,leftlabel,rightlabel)
                if gini_gain>max_ginigain:
                    max_ginigain=gini_gain
                    split_collum=i
                    wheretosplit=split_point
        return split_collum,wheretosplit
    def build_tree(self,data,label_collum=0,depth=0):    
        tree=treenode()
        if np.unique(data[:,label_collum]).size==1:
            tree.label=np.unique(data[:,label_collum]).item()
        else:
            features=np.delete(data,label_collum,axis=1)   
            split_collum,wheretosplit=self.Gini_split(data,label_collum)
            leftdata=data[features[:,split_collum]<wheretosplit]
            rightdata=data[features[:,split_collum]>=wheretosplit]
            if leftdata.shape[0]==data.shape[0] or rightdata.shape[0]==data.shape[0]:
                unique_elements, counts_elements = np.unique(data[:,label_collum], return_counts=True)
                tree.label=unique_elements[counts_elements.argmax()]
            else:
                if depth==self.max_depth:
                    unique_elements, counts_elements = np.unique(leftdata[:,label_collum], return_counts=True)
                    leftlabel=unique_elements[counts_elements.argmax()]
                    tree=treenode(None, None, lambda features: features[split_collum]<wheretosplit)
                    tree.left=treenode(label=leftlabel)
                    unique_elements, counts_elements = np.unique(rightdata[:,label_collum], return_counts=True)
                    rightlabel=unique_elements[counts_elements.argmax()]
                    tree.right=treenode(label=rightlabel)
                else:
                    tree=treenode(None, None, lambda features: features[split_collum]<wheretosplit)
                    depth=depth+1
                    tree.left=self.build_tree(leftdata,label_collum,depth)            
                    tree.right=self.build_tree(rightdata,label_collum,depth)          
        return tree
    def fit(self,test_data,tree):
        labels=[]
        for i in test_data:
            labels.append(tree.decide_label(i))
        return np.array(labels)
    def k_fold_fit(self,test_data,trees):
        labels=[]
        for i in test_data:
            labeli=[]
            for tree in trees:
                labeli.append(tree.decide_label(i))
            unique_elements, counts_elements = np.unique(labeli, return_counts=True)
            labels.append(unique_elements[counts_elements.argmax()])
        return np.array(labels)
            

In [215]:
def accuracy(predicted_labels, true_labels):
    return (np.array(predicted_labels)==np.array(true_labels)).sum()/predicted_labels.size

In [294]:
DT=DecisionTree(10)

In [295]:
tree=DT.build_tree(total_train_data)

In [296]:
predict_labels=DT.fit(test_X,tree)

In [297]:
accuracy(predict_labels, test_Y)

0.9208741522230596

In [298]:
def k_fold_crossvalidationDT(total_train_data,k=5):
    accuracy_list=[]
    models=[]
    DT=DecisionTree(10)
    for i in range(k):
        portion_size=int(total_train_data.shape[0]/k)
        validation=total_train_data[i*portion_size:(i+1)*portion_size]
        validation_X=validation[:,1:]
        validation_Y=validation[:,0]
        train=np.append(total_train_data[0:i*portion_size], total_train_data[(i+1)*portion_size:],axis=0)
        tree=DT.build_tree(train)
        predict_labels=DT.fit(validation_X,tree)
        acc=accuracy(predict_labels,validation_Y)
        accuracy_list.append(acc)
        models.append(tree)
    return accuracy_list, models

In [299]:
accuracy_list, models=k_fold_crossvalidationDT(total_train_data)

In [300]:
accuracy_list=np.array(accuracy_list)
print(accuracy_list)

[0.89443921 0.89915174 0.91705938 0.90763431 0.91423186]


In [301]:
print("The average accuracy of 5-fold crossvalidation Decision Tree on validation set is: ", accuracy_list.mean())

The average accuracy of 5-fold crossvalidation Decision Tree on validation set is:  0.9065032987747408


In [302]:
predict_labels=DT.k_fold_fit(test_X,models)

In [303]:
accuracy(predict_labels, test_Y)

0.9464958553127355

The accuracy on test set is 94.6% when 5-fold cross validation is used, whereas the accuracy on test set is 92.1% without corss validation. 